In [1]:
from models.nac import NAC
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset

#from models.nac import NeuralAccumulatorCell
from models.nalu import NeuralArithmeticLogicUnitCell
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
import os

In [34]:
def reportLoss(loss, epoch):
    print("epoch {},  \t loss {}".format(epoch, loss))
    
def train(model, optimizer, x_train, y_train, epochs, batch_size):
    
    for epoch in range(epochs):
    
        
        for batch in range(len(x_train) // batch_size):
            
            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size),:]

            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                break
                print("nan detected")
            
            loss.backward()
            optimizer.step()
        
        if loss != loss:
            break
        
        if epoch % 100 == 0: 
            pass
            #reportLoss(loss.data, epoch)
            
    return test(model,x_train,y_train)
        
def test(model, x_test, y_test):
    
    model.eval()
    output_test = model(x_test)
    loss = F.mse_loss(output_test, y_test)
    
    return loss

In [42]:
test_per_range = 2
sample_size = 100
set_size = 10000

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 200
batch_size = 64

#values = np.linspace(1,1000,10)
values = [(0,1.0),(-1.0,1.0)]
test_values = [(0,1.0),(-1.0,1.0),(0,100),(-100,100)]
train_loss = np.zeros((len(values),len(test_values),test_per_range))
test_loss = np.zeros((len(values),len(test_values),test_per_range))
print(values)
for j, value in tqdm(enumerate(values)):
    
    print("Train range", value)
    
    for k, test_value in enumerate(test_values):

        print("\tTest range", test_value)
        min_value = value[0]
        max_value = value[1]

        test_min_value = -10
        test_max_value = 10

        train_acc = 0
        test_acc = 0
        for i in range(test_per_range):

            model = NeuralArithmeticLogicUnitCell(in_dim, out_dim)
            optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

            x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, max_value,
                                                                        sample_size, set_size, boundaries = None)

            x_test, y_test, _ = generate_synthetic_selection_dataset(test_min_value, test_max_value,
                                                                        sample_size, 1, boundaries = boundaries)

            train_loss[j,k,i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

            test_loss[j,k,i]  = test(model, x_test, y_test)

            print("\t \tExp", i, " Train loss: ", train_loss[j,k,i], "\t Test loss", test_loss[j,k,i])


[(0, 1.0), (-1.0, 1.0)]


Train range (0, 1.0)
	Test range (0, 1.0)
	 	Exp 0  Train loss:  2.926751729859234e-09 	 Test loss 7988809728.0
	 	Exp 1  Train loss:  12.86323356628418 	 Test loss 139.9550323486328
	Test range (-1.0, 1.0)
	 	Exp 0  Train loss:  32.0213737487793 	 Test loss 379.19134521484375
	 	Exp 1  Train loss:  3.371293022524924e-09 	 Test loss inf
	Test range (0, 100)
	 	Exp 0  Train loss:  12.39040756225586 	 Test loss inf
	 	Exp 1  Train loss:  12.916696548461914 	 Test loss 173.10366821289062
	Test range (-100, 100)
	 	Exp 0  Train loss:  2.77052640914917 	 Test loss 1.688619073999944e+32
	 	Exp 1  Train loss:  58.18092346191406 	 Test loss 1004.892333984375
Train range (-1.0, 1.0)
	Test range (0, 1.0)
	 	Exp 0  Train loss:  9.103372278218558e-09 	 Test loss inf
	 	Exp 1  Train loss:  1.183763842504959e-08 	 Test loss 1.9404849710060536e+19
	Test range (-1.0, 1.0)
	 	Exp 0  Train loss:  7.623495612563147e-09 	 Test loss 4389760139264.0
	 	Exp 1  Train loss:  6.278896069744633e-09 	 Test loss i

In [13]:
test_per_range = 1
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

#values = np.linspace(1,1000,10)
values = [[0.0,1.0], 
          [-0.001,0.001],
          [0,100],
          [-100.0,100.0],
          [-1000.0,1000.0]]

for value in values:
    min_value, max_value=  value

    x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                sample_size, set_size, boundaries =boundaries)

    #x_test = x_test.type(torch.DoubleTensor)
    #y_test = y_test.type(torch.DoubleTensor)
    test_loss  = test(model, x_test, y_test)

    print(float(test_loss.data.numpy())/torch.max(x_test).numpy())


2.3853574531700008e-08
1.9732325351365785e-12
2.520233212882678e-06
2.1282146007719363e-07
2.0609418029654535e-06


In [5]:
np.savetxt("train_loss_exp4.csv", train_loss, delimiter=',', fmt='%2.2f')
np.savetxt("test_loss_exp3.csv", test_loss, delimiter=',', fmt='%2.2f')

np.savetxt("all_train_loss_exp3.csv", np.mean(train_loss<0.001,axis=1), delimiter=',', fmt='%2.2f')
np.savetxt("all_test_loss_exp3.csv", np.mean(test_loss<0.001,axis=1), delimiter=',', fmt='%2.2f')

In [36]:
from activationFunctions.Gumbel import gumbel_softmax
x = np.linspace(-10,10, 100)
taus = [0.05,0.5,0.8,1]
results = np.zeros((len(taus),len(x)))
for i, tau in enumerate(taus):
    for j,val in enumerate(x):
        val = val* torch.ones(1,2)

        y = gumbel_softmax(val, tau,)
        print(y)
        results[i, j] = y

    plt.plot(x, results[i, :])
    
plt.show()

tensor([[ 1.0000e+00,  1.4762e-08]])


ValueError: only one element tensors can be converted to Python scalars

In [32]:
results

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.